In [2]:
import duckdb

with duckdb.connect("data/sakila.duckdb") as conn:
    description = conn.sql("DESC;").df()

description

,database,schema,name,column_names,column_types,temporary
0,sakila,staging,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,sakila,staging,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,sakila,staging,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,sakila,staging,actor,"[actor_id, first_name, last_name, last_update,...","[DECIMAL(38,9), VARCHAR, VARCHAR, TIMESTAMP, V...",False
4,sakila,staging,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
5,sakila,staging,category,"[category_id, name, last_update, _dlt_load_id,...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
6,sakila,staging,city,"[city_id, city, country_id, last_update, _dlt_...","[BIGINT, VARCHAR, BIGINT, TIMESTAMP, VARCHAR, ...",False
7,sakila,staging,country,"[country_id, country, last_update, _dlt_load_i...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
8,sakila,staging,customer,"[customer_id, store_id, first_name, last_name,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, BI...",False
9,sakila,staging,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",False


In [3]:
dfs = {}

with duckdb.connect("data/sakila.duckdb") as conn:
    for name in description["name"]:
        if name[0] != "_":
            dfs[name] = conn.sql(f"FROM staging.{name};").df()
            duckdb.register(name, dfs[name])

dfs['actor'].head()

,actor_id,first_name,last_name,last_update,_dlt_load_id,_dlt_id
0,1.0,PENELOPE,GUINESS,2021-03-06 15:51:59,1764715358.6244173,6fSc6G4t1DiPZQ
1,2.0,NICK,WAHLBERG,2021-03-06 15:51:59,1764715358.6244173,uDVdDtKlKepz3w
2,3.0,ED,CHASE,2021-03-06 15:51:59,1764715358.6244173,2BFCklxajwwPaw
3,4.0,JENNIFER,DAVIS,2021-03-06 15:51:59,1764715358.6244173,Z0cB8ABLwQaETA
4,5.0,JOHNNY,LOLLOBRIGIDA,2021-03-06 15:51:59,1764715358.6244173,eoy7LC1CVCOf7A


In [4]:
duckdb.sql("DESC;").df()

,database,schema,name,column_names,column_types,temporary
0,temp,main,actor,"[actor_id, first_name, last_name, last_update,...","[DOUBLE, VARCHAR, VARCHAR, TIMESTAMP, VARCHAR,...",True
1,temp,main,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, ""NULL"", VARCHAR, BIGINT, VAR...",True
2,temp,main,category,"[category_id, name, last_update, _dlt_load_id,...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",True
3,temp,main,city,"[city_id, city, country_id, last_update, _dlt_...","[BIGINT, VARCHAR, BIGINT, TIMESTAMP, VARCHAR, ...",True
4,temp,main,country,"[country_id, country, last_update, _dlt_load_i...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",True
5,temp,main,customer,"[customer_id, store_id, first_name, last_name,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, BI...",True
6,temp,main,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",True
7,temp,main,film_actor,"[actor_id, film_id, last_update, _dlt_load_id,...","[BIGINT, BIGINT, TIMESTAMP, VARCHAR, VARCHAR]",True
8,temp,main,film_category,"[film_id, category_id, last_update, _dlt_load_...","[BIGINT, BIGINT, TIMESTAMP, VARCHAR, VARCHAR]",True
9,temp,main,inventory,"[inventory_id, film_id, store_id, last_update,...","[BIGINT, BIGINT, BIGINT, TIMESTAMP, VARCHAR, V...",True


In [5]:
duckdb.sql("FROM actor;").df()

,actor_id,first_name,last_name,last_update,_dlt_load_id,_dlt_id
0,1.0,PENELOPE,GUINESS,2021-03-06 15:51:59,1764715358.6244173,6fSc6G4t1DiPZQ
1,2.0,NICK,WAHLBERG,2021-03-06 15:51:59,1764715358.6244173,uDVdDtKlKepz3w
2,3.0,ED,CHASE,2021-03-06 15:51:59,1764715358.6244173,2BFCklxajwwPaw
3,4.0,JENNIFER,DAVIS,2021-03-06 15:51:59,1764715358.6244173,Z0cB8ABLwQaETA
4,5.0,JOHNNY,LOLLOBRIGIDA,2021-03-06 15:51:59,1764715358.6244173,eoy7LC1CVCOf7A
...,...,...,...,...,...,...
195,196.0,BELA,WALKEN,2021-03-06 15:52:00,1764715358.6244173,tk1XWKG4pu0rQg
196,197.0,REESE,WEST,2021-03-06 15:52:00,1764715358.6244173,Fyx2Z7aSNS1mlw
197,198.0,MARY,KEITEL,2021-03-06 15:52:00,1764715358.6244173,XtkEFWK6tQTILQ
198,199.0,JULIA,FAWCETT,2021-03-06 15:52:00,1764715358.6244173,LpGpgpxJEUDfjQ


In [6]:
import matplotlib

actor_films = duckdb.sql("""
    select * from film order by film_id limit 10;
           
""").df()

actor_films.head(10)

ax = actor_films.head(10).plot(
    kind="barh",
    x="actor",
    y="number_films",
    title="top 10 prolific actors",
    xlabel="number of films",
)
ax.invert_yaxis()

KeyError: 'actor'